# Task 4: Classification of tweets self-reporting potential COVID19 cases

In [1]:
!pip install -q transformers contractions imbalanced-learn ekphrasis

     |████████████████████████████████| 4.0 MB 25.5 MB/s 
     |████████████████████████████████| 80 kB 9.4 MB/s 
     |████████████████████████████████| 895 kB 52.0 MB/s 
     |████████████████████████████████| 596 kB 70.0 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 
     |████████████████████████████████| 287 kB 66.1 MB/s 
     |████████████████████████████████| 106 kB 65.8 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 


## 1. Import all the necessary libraries and data files

In [2]:
import numpy as np
import pandas as pd

import warnings
import torch
import torch.nn as nn
import time

from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertTokenizerFast
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=None

In [3]:
train_filename = "train.tsv"
val_filename = "valid.tsv"

In [40]:
# Load data
train = pd.read_csv(train_filename, sep="\t", names=["tweet_id", "user_id", "tweet", "label"])
validation = pd.read_csv(val_filename, sep="\t")
# print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

In [41]:
print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

Shape of training data is (6465, 4) and validation data is (716, 4)


In [42]:
# Train top 5 rows
train.head().style.set_caption("Task 4: Train dataset")

,tweet_id,user_id,tweet,label
0,1239172732690014208,2391447188,We’re parking at the airport and my mom rolled down the window to speak to an attendant and my dad immediately said “we have the coronavirus sir”,0
1,1223737201030246402,1200539436167159809,I really didn’t expect this will go wide this way. I hope safety & health for all people of #Chine & whole world. We are just trying to show some support & respect to them as much we can especially doctors who bravely facing the dirty #coronaVirus.,0
2,1239385333319389185,838382730,"For those who believe they are immortal and continue to go out to the park without paying attention to the order to remain at home, these are the x-rays of a 28-year-old boy intubated in the ICU in my hospital for #coronavirus. Hint: the lungs are black, white is pneumonia",1
3,1236209435241938945,780855138,My flight from Jordan back to the US stops in Paris 😂 will I be quarantined? Stay tuned to find out 😂😂 #coronavirus,0
4,1233855551605440514,337103373,I went to the movies and the air was on. Now I'm out to eat and Olive Garden has the air on. I see these establishments are doing their best to fight the coronavirus.,0


## 2. Prepare the data - Clean & Prepare for Model

In [43]:
# Drop unwanted columns
train.drop(['tweet_id', 'user_id'], axis=1, inplace=True)
validation.drop(['tweet_id', 'user_id'], axis=1, inplace=True)

In [44]:
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [46]:
# train
# train_data = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train]
# print(train_data)

In [45]:
train['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [48]:
# Train top 5 rows after pre-processing
train[['label','clean_tweets']].head()

,label,clean_tweets
0,0,we ’ re parking at the airport and my mom rolled down the window to speak to an attendant and my dad immediately said “ we have the coronavirus sir ”
1,0,i really didn ’ t expect this will go wide this way . i hope safety & health for all people of <hashtag> chine </hashtag> & whole world . we are just trying to show some support & respect to them as much we can especially doctors who bravely facing the dirty <hashtag> corona virus </hashtag> .
2,1,"for those who believe they are immortal and continue to go out to the park without paying attention to the order to remain at home , these are the x - rays of a <number> - year - old boy intubated in the <allcaps> icu </allcaps> in my hospital for <hashtag> coronavirus </hashtag> . hint : the lungs are black , white is pneumonia"
3,0,my flight from jordan back to the us stops in paris 😂 will i be quarantined ? stay tuned to find out 😂 😂 <hashtag> coronavirus </hashtag>
4,0,i went to the movies and the air was on . now i am out to eat and olive garden has the air on . i see these establishments are doing their best to fight the coronavirus .


In [49]:
# Define BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [50]:
# Tokenize train and validation data
train_enc = tokenizer.batch_encode_plus(train.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")
valid_enc = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")

In [51]:
train_enc.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
train_enc.input_ids.shape, train_enc.token_type_ids.shape, train_enc.attention_mask.shape

(torch.Size([20544, 128]), torch.Size([20544, 128]), torch.Size([20544, 128]))

In [52]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [53]:
def get_dataloader(encoding, target):
    data = (TensorDataset(encoding.input_ids, encoding.token_type_ids, encoding.attention_mask, target))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [55]:
train_dataloader = get_dataloader(train_enc, torch.tensor(train['label'].to_list()))
valid_dataloader = get_dataloader(valid_enc, torch.tensor(validation['label'].to_list()))

In [56]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in train_dataloader:
    input_ids, type_ids, attn_mask, target = batch
    print(input_ids.shape, type_ids.shape, attn_mask.shape, target.shape)
    break

torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32])


## 3. Model Building - BERT

In [57]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [58]:
class BERTclassifier(nn.Module):
    def __init__(self, transformer):
        super(BERTclassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(768, 2)
    
    def forward(self, ip_ids, type_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask, 
                              token_type_ids=type_ids)
        return  self.linear_layer(op["pooler_output"])

In [59]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [60]:
transformer = BertModel.from_pretrained("bert-base-uncased")
model = BERTclassifier(transformer).to(device)
print(f"The model has {count_parameter(model)} trainable parameters.")

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 109483778 trainable parameters.


In [61]:
# for name, param in model.named_parameters():
#     if "pooler" in name or "linear" in name:#or "layer.11" in name or "layer.10" in name or "linear" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
#   print(name, param.shape, param.requires_grad)

print(f'The model has {count_parameter(model):,} trainable parameters')

The model has 109,483,778 trainable parameters


In [62]:
# Define optimizer and 
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr = 2e-5)

#### Train BERT Model

In [63]:
def train_model(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    pred, target = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch

        optim.zero_grad()
        output = model(input_ids, type_ids, attn_mask)
        loss = criterion(output, y)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim.step()

        epoch_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred)

def evaluate(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    pred, target = list(), list()

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch
        
        with torch.no_grad():
            output = model(input_ids, type_ids, attn_mask)
            loss = criterion(output, y)
            
            epoch_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred), pred, target

In [64]:
best_valid_loss = float('inf')
total_train_loss, total_valid_loss = list(), list()

In [65]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train_model(model, train_dataloader)
    total_train_loss.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate(model, valid_dataloader)
    total_valid_loss.append(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_pred, best_target = pred, target
        torch.save(model.state_dict(), "model_least_loss.pt")
        print("Best Model Saved!!\n")
    elif epoch % 3 == 0:
        torch.save(model.state_dict(), "model_checkpoint" + str(epoch) + ".pt")
        print("Checkpoint Model Saved!\n")
    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.13it/s]
2it [00:01,  1.38it/s]
3it [00:02,  1.42it/s]
4it [00:02,  1.47it/s]
5it [00:03,  1.51it/s]
6it [00:04,  1.55it/s]
7it [00:04,  1.58it/s]
8it [00:05,  1.60it/s]
9it [00:05,  1.61it/s]
10it [00:06,  1.62it/s]
11it [00:07,  1.62it/s]
12it [00:07,  1.63it/s]
13it [00:08,  1.63it/s]
14it [00:08,  1.64it/s]
15it [00:09,  1.64it/s]
16it [00:10,  1.64it/s]
17it [00:10,  1.64it/s]
18it [00:11,  1.64it/s]
19it [00:11,  1.64it/s]
20it [00:12,  1.64it/s]
21it [00:13,  1.63it/s]
22it [00:13,  1.63it/s]
23it [00:14,  1.63it/s]
24it [00:15,  1.63it/s]
25it [00:15,  1.63it/s]
26it [00:16,  1.63it/s]
27it [00:16,  1.63it/s]
28it [00:17,  1.63it/s]
29it [00:18,  1.63it/s]
30it [00:18,  1.63it/s]
31it [00:19,  1.63it/s]
32it [00:19,  1.62it/s]
33it [00:20,  1.62it/s]
34it [00:21,  1.62it/s]
35it [00:21,  1.62it/s]
36it [00:22,  1.61it/s]
37it [00:23,  1.61it/s]
38it [00:23,  1.61it/s]
39it [00:24,  1.61it/s]
40it [00:24,  1.

Best Model Saved!!

Epoch: 01
Train Total Loss: 0.351 | Train F1 Score: 0.329
Valid Total Loss: 0.267 | Valid F1 Score: 0.651
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.51it/s]
2it [00:01,  1.46it/s]
3it [00:02,  1.47it/s]
4it [00:02,  1.47it/s]
5it [00:03,  1.46it/s]
6it [00:04,  1.46it/s]
7it [00:04,  1.47it/s]
8it [00:05,  1.46it/s]
9it [00:06,  1.47it/s]
10it [00:06,  1.47it/s]
11it [00:07,  1.46it/s]
12it [00:08,  1.46it/s]
13it [00:08,  1.46it/s]
14it [00:09,  1.46it/s]
15it [00:10,  1.46it/s]
16it [00:10,  1.46it/s]
17it [00:11,  1.46it/s]
18it [00:12,  1.45it/s]
19it [00:12,  1.46it/s]
20it [00:13,  1.46it/s]
21it [00:14,  1.46it/s]
22it [00:15,  1.46it/s]
23it [00:15,  1.46it/s]
24it [00:16,  1.46it/s]
25it [00:17,  1.46it/s]
26it [00:17,  1.46it/s]
27it [00:18,  1.46it/s]
28it [00:19,  1.46it/s]
29it [00:19,  1.46it/s]
30it [00:20,  1.46it/s]
31it [00:21,  1.46it/s]
32it [00:21,  1.46it/s]
33it [00:22,  1.46it/s]
34it [00:23,  1.47it/s]
35it [00:23,  1.46it/s]
36it [00:24,  1.46it/s]
37it [00:25,  1.47it/s]
38it [00:25,  1.47it/s]
39it [00:26,  1.47it/s]
40it [00:27,  1.47it/s]
41it [00:28,  1.47it/s]
42it 

Best Model Saved!!

Epoch: 02
Train Total Loss: 0.212 | Train F1 Score: 0.703
Valid Total Loss: 0.228 | Valid F1 Score: 0.738
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.53it/s]
2it [00:01,  1.48it/s]
3it [00:02,  1.47it/s]
4it [00:02,  1.48it/s]
5it [00:03,  1.47it/s]
6it [00:04,  1.47it/s]
7it [00:04,  1.48it/s]
8it [00:05,  1.47it/s]
9it [00:06,  1.47it/s]
10it [00:06,  1.48it/s]
11it [00:07,  1.48it/s]
12it [00:08,  1.48it/s]
13it [00:08,  1.48it/s]
14it [00:09,  1.48it/s]
15it [00:10,  1.48it/s]
16it [00:10,  1.48it/s]
17it [00:11,  1.48it/s]
18it [00:12,  1.47it/s]
19it [00:12,  1.47it/s]
20it [00:13,  1.47it/s]
21it [00:14,  1.48it/s]
22it [00:14,  1.48it/s]
23it [00:15,  1.48it/s]
24it [00:16,  1.47it/s]
25it [00:16,  1.47it/s]
26it [00:17,  1.48it/s]
27it [00:18,  1.47it/s]
28it [00:18,  1.47it/s]
29it [00:19,  1.47it/s]
30it [00:20,  1.47it/s]
31it [00:21,  1.47it/s]
32it [00:21,  1.47it/s]
33it [00:22,  1.47it/s]
34it [00:23,  1.47it/s]
35it [00:23,  1.47it/s]
36it [00:24,  1.47it/s]
37it [00:25,  1.47it/s]
38it [00:25,  1.47it/s]
39it [00:26,  1.47it/s]
40it [00:27,  1.47it/s]
41it [00:27,  1.47it/s]
42it 

Epoch: 03
Train Total Loss: 0.112 | Train F1 Score: 0.874
Valid Total Loss: 0.333 | Valid F1 Score: 0.713
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.51it/s]
2it [00:01,  1.49it/s]
3it [00:02,  1.48it/s]
4it [00:02,  1.48it/s]
5it [00:03,  1.47it/s]
6it [00:04,  1.47it/s]
7it [00:04,  1.47it/s]
8it [00:05,  1.47it/s]
9it [00:06,  1.47it/s]
10it [00:06,  1.48it/s]
11it [00:07,  1.48it/s]
12it [00:08,  1.47it/s]
13it [00:08,  1.47it/s]
14it [00:09,  1.47it/s]
15it [00:10,  1.48it/s]
16it [00:10,  1.48it/s]
17it [00:11,  1.47it/s]
18it [00:12,  1.47it/s]
19it [00:12,  1.47it/s]
20it [00:13,  1.48it/s]
21it [00:14,  1.48it/s]
22it [00:14,  1.48it/s]
23it [00:15,  1.48it/s]
24it [00:16,  1.49it/s]
25it [00:16,  1.49it/s]
26it [00:17,  1.48it/s]
27it [00:18,  1.48it/s]
28it [00:18,  1.48it/s]
29it [00:19,  1.48it/s]
30it [00:20,  1.48it/s]
31it [00:20,  1.48it/s]
32it [00:21,  1.47it/s]
33it [00:22,  1.48it/s]
34it [00:23,  1.48it/s]
35it [00:23,  1.47it/s]
36it [00:24,  1.48it/s]
37it [00:25,  1.47it/s]
38it [00:25,  1.47it/s]
39it [00:26,  1.47it/s]
40it [00:27,  1.47it/s]
41it [00:27,  1.46it/s]
42it 

Checkpoint Model Saved!

Epoch: 04
Train Total Loss: 0.058 | Train F1 Score: 0.944
Valid Total Loss: 0.458 | Valid F1 Score: 0.684
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.52it/s]
2it [00:01,  1.48it/s]
3it [00:02,  1.49it/s]
4it [00:02,  1.48it/s]
5it [00:03,  1.47it/s]
6it [00:04,  1.47it/s]
7it [00:04,  1.47it/s]
8it [00:05,  1.47it/s]
9it [00:06,  1.47it/s]
10it [00:06,  1.47it/s]
11it [00:07,  1.47it/s]
12it [00:08,  1.47it/s]
13it [00:08,  1.47it/s]
14it [00:09,  1.47it/s]
15it [00:10,  1.47it/s]
16it [00:10,  1.47it/s]
17it [00:11,  1.46it/s]
18it [00:12,  1.47it/s]
19it [00:12,  1.47it/s]
20it [00:13,  1.47it/s]
21it [00:14,  1.48it/s]
22it [00:14,  1.47it/s]
23it [00:15,  1.47it/s]
24it [00:16,  1.48it/s]
25it [00:16,  1.48it/s]
26it [00:17,  1.47it/s]
27it [00:18,  1.47it/s]
28it [00:19,  1.47it/s]
29it [00:19,  1.47it/s]
30it [00:20,  1.47it/s]
31it [00:21,  1.47it/s]
32it [00:21,  1.47it/s]
33it [00:22,  1.47it/s]
34it [00:23,  1.47it/s]
35it [00:23,  1.47it/s]
36it [00:24,  1.47it/s]
37it [00:25,  1.47it/s]
38it [00:25,  1.47it/s]
39it [00:26,  1.47it/s]
40it [00:27,  1.47it/s]
41it [00:27,  1.47it/s]
42it 

Epoch: 05
Train Total Loss: 0.034 | Train F1 Score: 0.972
Valid Total Loss: 0.521 | Valid F1 Score: 0.670
--------------------


In [66]:
print(classification_report(best_target, best_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       594
           1       0.74      0.74      0.74       122

    accuracy                           0.91       716
   macro avg       0.84      0.84      0.84       716
weighted avg       0.91      0.91      0.91       716



In [67]:
from google.colab import drive

In [68]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [70]:
from glob import glob
for filepath in glob("*.pt"):
    !cp -r $filepath /content/gdrive/My\ Drive/Colab\ Notebooks/
    time.sleep(10)